In [1]:
import pymongo
import pandas as pd
import numpy as np

import timeit
%load_ext Cython

In [2]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
mdf = client['DuProprio']['ListingDetails']
cursor = mdf.find({'results': {'$exists': True}})
df = pd.DataFrame(cursor)
df = df.drop(columns=['_id', 'link', 'description', 'results', 'listing_num', 'address', 'geo_index', 'price']).select_dtypes(include=['object']).apply(lambda x: np.array(x))

30

In [ ]:
from setuptools import setup
from Cython.Build import cythonize

setup(
    name='Pandas List Unpacker to One Hot Encoding',
    ext_modules=cythonize("pd_list_unpack_onehot.pyx"),
    zip_safe=False,
)

In [188]:
%%cython
from cpython cimport array
cimport numpy as np

import array 

cpdef unpack(np.ndarray a):
    cdef int shape = a.shape[0]
    cdef int i
    cdef int q
    for i in range(shape):
        if not type(a[i]) == float:
            for q in range(len(a[i])):
                a[i][q]


In [210]:
from numba import njit, vectorize
import numpy as np
@njit
def unpack_j(row):
    for i in range(len(row)):
        for q in range(len(row[i])):
            row[i][q]

In [199]:
def unpack_p(row):
    for i in row:
        if not isinstance(i, float):
            for q in i:
                if not isinstance(q, float):
                    q

In [141]:
type(array[0])

numpy.str_

In [189]:
%%timeit
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=4) as exec:
    exec.submit(unpack, df.select_dtypes(include=['object'].apply(lambda x: np.array(x)).to_numpy()))

33.4 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [213]:
# %%timeit

# from concurrent.futures import ThreadPoolExecutor
array = np.array(['Apple', 'Sauce', 'Boss'])
# arrayb = [x.encode('utf-8') for x in array]
%timeit _ = unpack_j(df.Bathroom.apply(lambda x: np.array(x)).to_list())

TypeError: can't unbox heterogeneous list: array([unichr x 15], 1d, C) != array([unichr x 17], 1d, C)

In [200]:
array = np.array(['Apple', 'Sauce', 'Boss'])
%timeit _ = unpack_p(df.Bathroom.apply(lambda x: np.array(x)).to_numpy())

35.9 ms ± 2.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [194]:
%%cython

def one_hot(frame, categ_cols):
    for col in categ_cols:
        if not col in ['last_modified', 'geo_index']:
            frame = frame.apply(lambda x: unpack_list(x, col), axis=1)
            frame = frame.drop(columns=[col])
        return frame


Error compiling Cython file:
------------------------------------------------------------
...

def one_hot(frame, categ_cols):
    for col in categ_cols:
        if not col in ['last_modified', 'geo_index']:
            frame = frame.apply(lambda x: unpack_list(x, col), axis=1)
                                         ^
------------------------------------------------------------

/Users/mathewzaharopoulos/.ipython/cython/_cython_magic_7cf2a349c62167e27d83dabc094fa907.pyx:5:42: undeclared name not builtin: unpack_list


In [223]:
df = df.drop(columns=['_id', 'link', 'description', 'results', 'listing_num', 'address', 'geo_index']).select_dtypes(include=['object']).apply(lambda x: np.array(x))

In [234]:
df['county']

0                           St-Amable
1                             Bromont
2                           Val-David
3                             Ste-Foy
4       Drummondville (St-Nicéphore)
                    ...              
8313                       Natashquan
8314                   Aston Jonction
8315                       Les Saules
8316                        St-Lazare
8317           St-Joachim-De-Shefford
Name: county, Length: 8318, dtype: object

In [232]:
df.shape[1]
df.shape[0]

8318

In [207]:
%%cython

cimport numpy as np
# from collections cimport defaultdict
import pandas as pd

cpdef unpack_list(np.ndarray col):
    cdef int length = col.shape[0]
        
    cdef int i
    cdef int q
    
    ev = None
    for i in range(length):
        out = {}
        cell = col[i]
        if type(cell) == list:
            for q in range(len(cell)):
                out.update({cell[q]:1})
            if ev is None:
                ev = pd.Series(out)
            else:
                pd.concat([ev, pd.Series(out)])
    return ev

            
            


        


In [307]:
type(df.shape[0]-1)

int

In [139]:
# %%timeit
from concurrent.futures import ThreadPoolExecutor

t = transform_pd_columns(df.shape[0], df.shape[1])
with ThreadPoolExecutor(max_workers=4) as exec:
    exec.submit(t.make_dict, df)

In [208]:
# %%timeit
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=4) as exec:
    final = exec.submit(df.apply(lambda x: unpack_list(x.to_numpy()), axis=0))
#     print(final.result())

In [211]:
df.apply(lambda x: unpack_list(x.to_numpy()), axis=1)

,Acrylique resin,Adex,Aggregate,Air exchanger,Alarm system,Aluminium Siding,Asbestos,Asphalt,Attached,B/I Microwave,...,Walk-in closet,Water Access,Water view,Waterfront,Window coverings,Windows,With electrical outlet,Wood,Wood stove,Wooden cabinets
0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
8315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
